<a href="https://colab.research.google.com/github/pmckim1/Capstone/blob/main/Capstone_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# use high-ram to speed up processing on collab 
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.7 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


In [2]:
# install chart_studio for visualizations 
!pip install chart_studio

     |████████████████████████████████| 71kB 4.5MB/s 


In [3]:
# libraries 
import numpy as np
from google.colab import drive
import json 
import sys
import pandas as pd
import datetime as dt
import plotly.express as px
from datetime import timezone
import plotly.graph_objects as go
import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls
import matplotlib as plt
import networkx as nx
pd.set_option('display.max_columns', None)
# Mount Google Drive
drive.mount('/content/drive')

# Get the absolute path of the current folder
abspath_curr = '/content/drive/My Drive/capstone/capstone/Data/Guardian/'


Mounted at /content/drive


In [4]:
# plotly chart credentials 
# plotly chart
user_name="mckim1"
apikey="TzHNPfnv0sAafxwEKbik"

chart_studio.tools.set_credentials_file(username=user_name, api_key=apikey)

In [5]:
# load in data 
df = pd.read_csv(abspath_curr +'ready_for_bert.csv', header=0,  parse_dates=['pub_date_dto'])
# drop unneeded columns 
df=df.drop(["Unnamed: 0", "headline_tokens","head_ents_label", "headline_lemma", "headline_pos", "headline_ents", "head_ents_label", "timeset"], axis=1)


#%%
print(df.info())
#%%
print(len(df))
df.dropna()
                    
print(len(df))

df["v_hier_infomap_comm"] = df["v_hier_infomap_comm"].astype(str)


df["headline"] = df["headline"].astype(str)
#print(len(df))
#print(df.head())
#print(df.sectionname.value_counts())

# sort dates from first to last 
df=df.sort_values(by='pub_date_dto')
# clean up the data by remove very short articles, visual articles
df["text"]=df['text'].astype(str)
top_story = df[df["text"].str.contains("Top story")]
df = df[~df["text"].str.contains("Top story")]
df = df[~df["headline"].str.contains("a visual guide")]
happened = df[df["headline"].str.contains("as it happened")]
df = df[~df["headline"].str.contains("as it happened")]
print(len(df))
df["text"] = df["text"].astype(str)
df["headline"] = df["headline"].astype(str)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98623 entries, 0 to 98622
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   t                    98559 non-null  float64       
 1   pub_date             98559 non-null  object        
 2   sectionid            98559 non-null  object        
 3   sectionname          98559 non-null  object        
 4   headline             98559 non-null  object        
 5   text                 98559 non-null  object        
 6   pub_date_dto         98559 non-null  datetime64[ns]
 7   v_id                 98623 non-null  object        
 8   Id                   98623 non-null  object        
 9   Label                98623 non-null  object        
 10  v_name               98623 non-null  object        
 11  v_hier_infomap_comm  98623 non-null  object        
 12  v_url                98623 non-null  object        
dtypes: datetime64[ns](1), float64(1

# Exploratory Data Analysis  
This exploration will invesigate the data exported after the algorithmn successfully grouped articles following the same event together. 



In [6]:
# missing data
def missing_values_table(df):
    # Utility function, identify missing data and show percentages.
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
    print("Your selected dataframe has " + str(df.shape[1]) + " columns.\nThere are " + str(mis_val_table_ren_columns.shape[0]) + " columns that have missing values.")
    return mis_val_table_ren_columns

missing_values_table(df)

Your selected dataframe has 13 columns.
There are 5 columns that have missing values.


,Missing Values,% of Total Values
t,64,0.1
pub_date,64,0.1
sectionid,64,0.1
sectionname,64,0.1
pub_date_dto,64,0.1


We see one column is entirely empty so we will drop it. We will also drop the rows missing data as they are minimal  

In [7]:
# drop the missing values 
df=df.dropna(axis = 0, how ='any') 


In [8]:
# confirm values are not missing 
missing_values_table(df)

Your selected dataframe has 13 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [9]:
# get article cluster counts 
df['cluster_counts'] = df.groupby('v_hier_infomap_comm')['v_hier_infomap_comm'].transform('count')

df["cluster_counts"] = df["cluster_counts"].astype(int)

In [10]:
# cluster versus non cluster 
def cluster_or_non(row):
    if row > 1:
        return True
    return False
df['cluster_or_non'] = df.apply(lambda row: cluster_or_non(row["cluster_counts"]), axis=1)

## Bar Graph of Amount of articles in each section 

Here we see that World News has the most followed by US News and Political News. It is important to note that in this context, Political News refers to UK political coverage. 

In [11]:
# add count column for easy aggregation 
df["count"]=1



df_copy = df.copy(deep=True)
def bar_graph_group_values(df, df_column, New_Name, Title):
    dfsn = df.groupby(df_column).count().reset_index()
    dfsn = dfsn.rename(columns={df_column: New_Name})
    fig = px.bar(dfsn, x=New_Name, y= "count", title= Title, color_discrete_sequence =['#0000FF']*len(dfsn))
    fig.show()
    return fig
fig=bar_graph_group_values(df_copy, "sectionname", "Section", "Amount of Articles in Each Section")



In [12]:
py.plot(fig,filename="article_each_sec",auto_open=False)

'https://plotly.com/~mckim1/35/'

## Articles published per year 
More articles were published in 2016 & 2020 which were both election years in the US. 2021 is incomplete data so those numbers are artificially low

In [13]:
df['pub_date_dto'] = pd.to_datetime(df['pub_date_dto'], format='%Y-%m-%d')

In [14]:
df_year=df.groupby(df['pub_date_dto'].dt.year)['count'].agg(['sum']).reset_index() 
df_year = df_year.rename(columns={"pub_date_dto": "Year", "sum": "Count"})
print(df_year.head())

   Year  Count
0  2016  20692
1  2017  16783
2  2018  17002
3  2019  17522
4  2020  21302


In [15]:

fig = px.bar(df_year, x="Year", y= "Count", title= "Articles Published Per Year", color_discrete_sequence =['#0000FF']*len(df_year))
fig.show()
py.plot(fig,filename="year_article_graph",auto_open=False)

'https://plotly.com/~mckim1/37/'

# Line Graph of Articles Published overtime by Section  
We can see the follwoing spikes 
*   World News in March 2020 (likely coinciding with Covid related coverage) 
*   Politics in June 2016 (likely coinciding with Brexit)
*   US News in November 2016 (likely coinciding with the US presidential election) 
*   Global Development displays changes in amount of articles overtime but does not have any disproportional spikes








In [16]:
# line graph of articles published each day 
df_line=df.groupby(["sectionname",'pub_date_dto']).count().reset_index()
fig = px.line(df_line, x="pub_date_dto", y="count", title= 'Total Articles Published Over Time By Section', color="sectionname")
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=7, label="1week", step="day", stepmode="todate"),
            dict(count=1, label="1d", step="month", stepmode="todate"),
            dict(count=3, label="1m", step="month", stepmode="todate"),
            dict(step="all")
        ])
    ))
fig.update_layout(xaxis_title='Date',
                   yaxis_title='Articles')   
fig.show()
py.plot(fig,filename="article_each_sec_line",auto_open=False)

'https://plotly.com/~mckim1/39/'

# Bar Graph of Amount of Articles in Narrative Chains


In [21]:

df["cluster_counts"] = df["cluster_counts"].astype(int)
### *** need this graph double checked 
df_copy = df.copy(deep=True)
df_copy = df_copy[df_copy['cluster_counts'].between(2, 10)]

 
df_clusters = df_copy.groupby('v_hier_infomap_comm').count()


def bar_graph_cluster_values(df, df_column, New_Name, Title):
    dfsn = df.groupby(df_column).count().reset_index()
    dfsn = dfsn.rename(columns={df_column: New_Name})
    fig = px.bar(dfsn, x=New_Name, y= "headline", title= Title)
    fig.show()
    return fig1
fig1=bar_graph_group_values(df_clusters, "cluster_counts", "Clusters", "Amount of Clusters with 2-10 Articles")
py.plot(fig1,filename="article_cluster_1",auto_open=False)




'https://plotly.com/~mckim1/45/'

In [18]:
df_copy = df.copy(deep=True)
df_copy = df_copy[df_copy['cluster_counts'].between(11, 30)]

df_clusters = df_copy.groupby('v_hier_infomap_comm').count()

fig2=bar_graph_group_values(df_clusters, "cluster_counts", "Clusters", "Amount of Clusters with 11-30 Articles")
py.plot(fig2,filename="article_cluster_2",auto_open=False)



'https://plotly.com/~mckim1/41/'

In [19]:
df_copy = df.copy(deep=True)
df_copy = df_copy[df_copy['cluster_counts'].between(31, 99)]
print(len(df_copy))

df_clusters = df_copy.groupby('v_hier_infomap_comm').count()



fig3=bar_graph_group_values(df_clusters, "cluster_counts", "Clusters", "Amount of Clusters with 31+ Articles")
py.plot(fig3,filename="article_cluster_3",auto_open=False)


1440


'https://plotly.com/~mckim1/43/'

## Pie Chart of Articles Assigned to Narrative Chain Clusters versus Assigned to Clusters of One Article

More articles were not a part of a narrative chain. However, a large percentage of articles were members of a narrative chain community. This tracks with the findings from the Nichols/Bright algorithm 


In [22]:





# get percent of SPEED accidents compared to overall accidents 
df_pie = df.groupby(["cluster_or_non"]).sum().reset_index()

fig = px.pie(df_pie, values='count', names='cluster_or_non', color_discrete_sequence=px.colors.diverging.balance, title="Percent of Articles Assigned to Narrative Chain Clusters")
fig.show()
py.plot(fig,filename="EDA_Pie_Chart",auto_open=False)


'https://plotly.com/~mckim1/47/'